In [1]:
import os,glob
import datetime
import xarray as xr
import netCDF4 as nc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp2d

In [2]:
def remove_duplicates(input_list):
    new_list = []
    for element in input_list:
        if element not in new_list:
            new_list.append(element)
    return new_list

In [3]:
def convert_sst(sst_time):
    sst_time1 = []
    for j in range(len(sst_time)):
        one = str(sst_time[j]).split("T")
        two = one[0]
        three = one[-1].split(".")[0]
        four = f"{two} {three}" # 格式为2019-07-11 06:00:00
        sst_time1.append(four)  
    
    return sst_time1

In [4]:
# 加载hycom文件
# folder_path = "/home/ly/xia_datan/hycom_surface_data"
folder_path = "/data3/hycom_surface_data"
filenames = glob.glob(os.path.join(folder_path,"*nc"))
filenames = sorted(filenames)

In [5]:
# 为了方便查找对应的时间，只提取月日
def parttime(filenames):
    part_filenames = []
    for i in range(len(filenames)):
        one = filenames[i].split(".")[0][-4:]
        part_filenames.append(one)
    return part_filenames

In [6]:
# 按照hycom的规矩记录时间点
start_time = datetime.datetime(2000,1,1)
now_time = datetime.datetime.now() 

total_seconds = (now_time - start_time).total_seconds()
total_hour = int(total_seconds / 3600)

# 建立对应的时间戳，方便查找
index_times = range(0,total_hour,3) # 3是因为hycom数据是3小时一条
name_times = []
for i in range(len(index_times)):
    one_time = start_time + datetime.timedelta(hours=index_times[i])
    one_time = one_time.strftime("%Y-%m-%d %H:%M:%S")
    # one_time = datetime.datetime.strftime(one_time,"%Y-%m-%d %H:%M:%S")
    name_times.append(one_time)

# print(name_times)

In [7]:
# 读取预处理好的轨迹数据
# bouydata = pd.read_csv("/home/ly/xia_datan/wangying/trace/bouydata_3.csv")
bouydata = pd.read_csv("/home/ly/xia_datan/bouydataTrace/data/bouydata_3_1.csv")
bouydata['time'] = pd.to_datetime(bouydata['time'])

clean_id = remove_duplicates(bouydata["id"]) # 浮标id
first_index = [] # 每台浮标的第一个编号
last_index = [] # 每台浮标的最后一个编号

for i in range(len(clean_id)):
    index = bouydata.index[bouydata["id"]==clean_id[i]].to_list()
    first_index.append(index[0])
    last_index.append(index[-1])

bouydata
# print(len(first_index))

,id,time,latitude,longitude,sst,VE,VN,SPD,VAR.LAT,VAR.LON,VAR.TEMP
0,92953,2010-12-22 12:00:00,17.978,124.963,27.640,-9.437,16.664,19.150,0.000005,0.000009,0.001766
1,92953,2010-12-22 18:00:00,17.996,124.964,27.640,-5.154,6.442,8.250,0.000006,0.000011,0.001685
2,92953,2010-12-23 00:00:00,18.004,124.942,27.671,-18.115,-2.556,18.295,0.000018,0.000035,0.001760
3,92953,2010-12-23 06:00:00,17.986,124.891,27.639,-33.065,10.778,34.777,0.000004,0.000008,0.001640
4,92953,2010-12-23 12:00:00,18.046,124.807,27.501,-21.919,30.386,37.467,0.000007,0.000014,0.002087
...,...,...,...,...,...,...,...,...,...,...,...
107546,300534064920860,2023-10-13 12:00:00,21.221,111.331,27.962,-2.695,3.717,4.592,0.000002,0.000004,0.001684
107547,300534064920860,2023-10-13 18:00:00,21.219,111.327,27.754,-5.732,0.897,5.801,0.000002,0.000004,0.001684
107548,300534064920860,2023-10-14 00:00:00,21.225,111.308,27.740,-5.591,0.628,5.626,0.000002,0.000004,0.001684
107549,300534064920860,2023-10-14 06:00:00,21.221,111.304,27.896,-1.006,2.758,2.936,0.000002,0.000004,0.001684


In [8]:
print(len(first_index))

578


In [9]:
# 随便寻找一个分辨率是0.04且数据没有缺失的文件，找到标准的lat和lon,为了插值用
std_filename = "/data3/hycom_surface_data/hycom_surface_20201027.nc"
std_data = xr.open_dataset(std_filename)
std_lat = std_data["lat"]
std_lon = std_data["lon"]
# print(std_lon.values)
# print(std_lat.values)
# 由于depth只取表层，所以这里不做考虑
# print(std_data["lon"].values.size)
print(len(std_lat))
print(len(std_lon))

926
338


In [11]:
sst_data = []
sal_data = []

u_data = []
v_data = []

# 气候态
missing_values_sst = {} # sst的气候态
missing_values_sal = {}
missing_values_u = {}
missing_values_v = {}

for k in range(len(first_index)):
# for  k in range(1,2):
    print(f"------------------------{k}-----------------------")
#     for i in range(first_index[k]+88,first_index[k]+88+1):
    for i in range(first_index[k],last_index[k]+1):
        print(i)
        one_bouydata = bouydata.loc[i]
        
        lat = one_bouydata.latitude
        lon = one_bouydata.longitude
        date_time = str(one_bouydata.time)
        bouydata_id = one_bouydata.id
        # print(bouydata_id)
        # print(date_time)
        
        # 找到hycom文件所在的位置
        filename = str(date_time).split(" ")[0].split("-")
        filename = "".join(filename)
        filename = f"/hycom_surface_{filename}.nc"
        hycom_filename = f"{folder_path}{filename}"

        # 由于数据不全导致的问题，这里随便放一个文件进去就行，不全的数据会用气候态数据补全
        try:
            hycom_data = xr.open_dataset(hycom_filename)
        except:
            hycom_data = xr.open_dataset("/data3/hycom_surface_data/hycom_surface_20201027.nc")
        # print(hycom_data)

        # 获取sst
        sel_hycom_sst = hycom_data["water_temp"]
        # sel_hycom_sst = sel_hycom_sst[:,:,::5,::5]
        # print(sel_hycom_sst)

        # 获取sal
        sel_hycom_sal = hycom_data["salinity"]
        # sel_hycom_sal = sel_hycom_sal[:,:,::5,::5]

        # 获取u
        sel_hycom_u = hycom_data["water_u"]
        # sel_hycom_u = sel_hycom_u[:,:,::5,::5]

        # 获取v
        sel_hycom_v = hycom_data["water_v"]
        # sel_hycom_v = sel_hycom_v[:,:,::5,::5]

        # 取的位置信息,深度信息,时间信息  由于文件一样，所以取sst的作为标准
        # print(sel_hycom_sst["time"].values)
        sel_hycom_loc = sel_hycom_sst.sel(lat=lat, lon=lon,method='nearest')
        # print(sel_hycom_loc)
        sel_depth = sel_hycom_sst["depth"].values
        sel_time = sel_hycom_sst["time"].values
        sel_time = convert_sst(sel_time) # 修改sel_time的格式
        
        # 划定范围
        rangepick = 0.5
        lat_0 = sel_hycom_loc.lat - rangepick
        lat_1 = sel_hycom_loc.lat + rangepick
        lon_0 = sel_hycom_loc.lon - rangepick
        lon_1 = sel_hycom_loc.lon + rangepick
        
        # print(lat_0.values, lat_1.values, lon_0.values, lon_1.values)
        # sst
        try:
            # a = sel_hycom_sst.loc[date_time,sel_depth[0],lat_0:lat_1,lon_0:lon_1] # 这一步只是为了先try出去
            
            # 对数据进行插值
            if sel_hycom_sst["lat"].values.size == len(std_lat): # 数据的分辨率和需要插值的分辨率相同
                a = sel_hycom_sst.loc[date_time,sel_depth[0],lat_0:lat_1,lon_0:lon_1]
                # 对nan数据进行插值
                a = pd.DataFrame(a.values)
                a.interpolate(method="linear",inplace=True)
                a.fillna(a.mean(),inplace=True)
                a.fillna(method="bfill",inplace=True,axis=1)
                a.fillna(method="ffill",inplace=True,axis=1)
                
                a = np.array(a).flatten()
            else:
                # print(sel_hycom_sst["time"].values)
                a = sel_hycom_sst.loc[date_time,sel_depth[0]]
                # print(a)
                a = a.interp(lon=std_lon,lat=std_lat)
                a = a.loc[lat_0:lat_1,lon_0:lon_1]
                # print(a.values)
                
                a = pd.DataFrame(a.values)
                a.interpolate(method="linear",inplace=True)
                a.fillna(a.mean(),inplace=True)
                a.fillna(method="bfill",inplace=True,axis=1)
                a.fillna(method="ffill",inplace=True,axis=1)
                # print(a.shape) # (25, 13)
                # print(a.values)
                a = np.array(a).flatten()  
                # print(len(a))
                # print(a)
        except:
            # print(str(date_time))
            miss_value_key = [str(date_time),np.round(lat_0.values,8),np.round(lat_1.values,8),np.round(lon_0.values,8),np.round(lon_1.values,8)]
            miss_value_key = tuple(miss_value_key)
            if miss_value_key in missing_values_sst: # 键值对中有这个值
                a = missing_values_sst.get(miss_value_key)
            else: # 键值对中没有这个值
                # 抽取月日
                month_day = date_time.split(" ")[0].split("-")
                month_day1 = f"{month_day[1]}{month_day[2]}" # 格式为0711这种
                month_day2 = f"{month_day[1]}-{month_day[2]}" # 格式为07-11这种
                # 抽取时间 格式为06:00:00这种
                hours = date_time.split(" ")[-1]

                # 查找对应的hycom文件
                part_filenames = parttime(filenames)
                indeices = [j for j, name in enumerate(part_filenames) if month_day1 in name] # indeices最终为一个数组表明所查找的文件在part_fimenames中的第几个位置   part_fimenames和filenames的大小是一样的
                # print(indeices)
                # 存储同一月日时的文件 这里不包含年是因为要靠年去区分
                one_datas = []
                for indeic in indeices: # 只是为了和indeics作区分，并没有写成index的形式
                    name_file = filenames[indeic]
                    # print(name_file)
                    hycom_data1 = xr.open_dataset(name_file)
                    hycom_data_sst1 = hycom_data1["water_temp"]
                    hycom_time1 = hycom_data1["time"].values # 标准格式
                    hycom_time2 = convert_sst(hycom_time1)

                    # 查找该文件是否存在这个时刻的数据
                    one_1 = [j for j, name in enumerate(hycom_time2) if hours in name]
                    one_2 = [j for j, name in enumerate(hycom_time2) if month_day2 in name]
                    one = list(filter(lambda x:x in one_1,one_2))

                    if one == []: # 没有这个时刻的数据，跳过
                        continue
                    else:
                        # 需要先向网格中进行插值然后再将数据append到one_datas中
                        if hycom_data_sst1["lat"].values.size == len(std_lat):
                            aa = hycom_data_sst1.loc[hycom_time1[one[0]],sel_depth[0],lat_0:lat_1,lon_0:lon_1]

                            aa = pd.DataFrame(aa.values)
                            aa.interpolate(method="linear",inplace=True)
                            aa.fillna(aa.mean(),inplace=True)
                            aa.fillna(method="bfill",inplace=True,axis=1)
                            aa.fillna(method="ffill",inplace=True,axis=1)

                            aa = np.array(aa).flatten()
                            # print(aa)
                        else:
                            aa = hycom_data_sst1.loc[hycom_time1[one[0]],sel_depth[0]]
                            aa = aa.interp(lon=std_lon,lat=std_lat)
                            aa = aa.loc[lat_0:lat_1,lon_0:lon_1]

                            aa = pd.DataFrame(aa.values)
                            aa.interpolate(method="linear",inplace=True)
                            aa.fillna(aa.mean(),inplace=True)
                            aa.fillna(method="bfill",inplace=True,axis=1)
                            aa.fillna(method="ffill",inplace=True,axis=1)

                            aa = np.array(aa).flatten()
                        one_datas.append(aa)

                one_datas = np.array(one_datas)
                one_datas_mean = one_datas.mean(axis=0)

                missing_values_sst[miss_value_key] = one_datas_mean
                # print(one_datas_mean)
                # a = pd.DataFrame(one_datas_mean.values)
                # a.fillna(a.mean(),inplace=True)
                # a.fillna(method="bfill",inplace=True,axis=1)

                a = np.array(one_datas_mean).flatten() 
        b = np.array([str(bouydata_id),date_time])
        b = np.hstack((b,a)) # 第一个为浮标编号，第二为时间
        # print(b)
        sst_data.append(b)


        # sal
        try:
            # a = sel_hycom_sst.loc[date_time,sel_depth[0],lat_0:lat_1,lon_0:lon_1] # 这一步只是为了先try出去

            # 对数据进行插值
            if sel_hycom_sal["lat"].values.size == len(std_lat): # 数据的分辨率和需要插值的分辨率相同
                a = sel_hycom_sal.loc[date_time,sel_depth[0],lat_0:lat_1,lon_0:lon_1]

                a = pd.DataFrame(a.values)
                a.interpolate(method="linear",inplace=True)
                a.fillna(a.mean(),inplace=True)
                a.fillna(method="bfill",inplace=True,axis=1)
                a.fillna(method="ffill",inplace=True,axis=1)

                a = np.array(a).flatten()
            else:
                # print(sel_hycom_sst["time"].values)
                a = sel_hycom_sal.loc[date_time,sel_depth[0]]
                # print(a)
                a = a.interp(lon=std_lon,lat=std_lat)
                a = a.loc[lat_0:lat_1,lon_0:lon_1]
                # print(a.values)

                a = pd.DataFrame(a.values)
                a.interpolate(method="linear",inplace=True)
                a.fillna(a.mean(),inplace=True)
                a.fillna(method="bfill",inplace=True,axis=1)
                a.fillna(method="ffill",inplace=True,axis=1)

                # print(a.shape) # (25, 13)
                # print(a.values)
                a = np.array(a).flatten()  
                # print(len(a))
                # print(a)
        except:
            # print(str(date_time))
            miss_value_key = [str(date_time),np.round(lat_0.values,8),np.round(lat_1.values,8),np.round(lon_0.values,8),np.round(lon_1.values,8)]
            miss_value_key = tuple(miss_value_key)
            if miss_value_key in missing_values_sal: # 键值对中有这个值
                a = missing_values_sal.get(miss_value_key)
            else: # 键值对中没有这个值
                # 抽取月日
                month_day = date_time.split(" ")[0].split("-")
                month_day1 = f"{month_day[1]}{month_day[2]}" # 格式为0711这种
                month_day2 = f"{month_day[1]}-{month_day[2]}" # 格式为07-11这种
                # 抽取时间 格式为06:00:00这种
                hours = date_time.split(" ")[-1]

                # 查找对应的hycom文件
                part_filenames = parttime(filenames)
                indeices = [j for j, name in enumerate(part_filenames) if month_day1 in name] # indeices最终为一个数组表明所查找的文件在part_fimenames中的第几个位置   part_fimenames和filenames的大小是一样的

                # 存储同一月日时的文件 这里不包含年是因为要靠年去区分
                one_datas = []
                for indeic in indeices: # 只是为了和indeics作区分，并没有写成index的形式
                    name_file = filenames[indeic]
                    # print(name_file)
                    hycom_data1 = xr.open_dataset(name_file)
                    hycom_data_sst1 = hycom_data1["salinity"]
                    hycom_time1 = hycom_data1["time"].values # 标准格式
                    hycom_time2 = convert_sst(hycom_time1)

                    # 查找该文件是否存在这个时刻的数据
                    one_1 = [j for j, name in enumerate(hycom_time2) if hours in name]
                    one_2 = [j for j, name in enumerate(hycom_time2) if month_day2 in name]
                    one = list(filter(lambda x:x in one_1,one_2))

                    if one == []: # 没有这个时刻的数据，跳过
                        continue
                    else:
                        # 需要先向网格中进行插值然后再将数据append到one_datas中
                        if hycom_data_sst1["lat"].values.size == len(std_lat):
                            aa = hycom_data_sst1.loc[hycom_time1[one[0]],sel_depth[0],lat_0:lat_1,lon_0:lon_1]

                            aa = pd.DataFrame(aa.values)
                            aa.interpolate(method="linear",inplace=True)
                            aa.fillna(aa.mean(),inplace=True)
                            aa.fillna(method="bfill",inplace=True,axis=1)
                            aa.fillna(method="ffill",inplace=True,axis=1)

                            aa = np.array(aa).flatten()
                        else:
                            aa = hycom_data_sst1.loc[hycom_time1[one[0]],sel_depth[0]]
                            aa = aa.interp(lon=std_lon,lat=std_lat)
                            aa = aa.loc[lat_0:lat_1,lon_0:lon_1]

                            aa = pd.DataFrame(aa.values)
                            aa.interpolate(method="linear",inplace=True)
                            aa.fillna(aa.mean(),inplace=True)
                            aa.fillna(method="bfill",inplace=True,axis=1)
                            aa.fillna(method="ffill",inplace=True,axis=1)

                            aa = np.array(aa).flatten()
                        one_datas.append(aa)

                one_datas = np.array(one_datas)
                one_datas_mean = one_datas.mean(axis=0)

                missing_values_sal[miss_value_key] = one_datas_mean
                # print(one_datas_mean)
                # a = pd.DataFrame(one_datas_mean.values)
                # a.fillna(a.mean(),inplace=True)
                # a.fillna(method="bfill",inplace=True,axis=1)

                a = np.array(one_datas_mean).flatten()  
        b = np.array([str(bouydata_id),date_time])
        b = np.hstack((b,a)) # 第一个为浮标编号，第二为时间
        sal_data.append(b)


        # u
        try:
            # a = sel_hycom_sst.loc[date_time,sel_depth[0],lat_0:lat_1,lon_0:lon_1] # 这一步只是为了先try出去

            # 对数据进行插值
            if sel_hycom_u["lat"].values.size == len(std_lat): # 数据的分辨率和需要插值的分辨率相同
                a = sel_hycom_u.loc[date_time,sel_depth[0],lat_0:lat_1,lon_0:lon_1]

                a = pd.DataFrame(a.values)
                a.interpolate(method="linear",inplace=True)
                a.fillna(a.mean(),inplace=True)
                a.fillna(method="bfill",inplace=True,axis=1)
                a.fillna(method="ffill",inplace=True,axis=1)

                a = np.array(a).flatten()
            else:
                # print(sel_hycom_sst["time"].values)
                a = sel_hycom_u.loc[date_time,sel_depth[0]]
                # print(a)
                a = a.interp(lon=std_lon,lat=std_lat)
                a = a.loc[lat_0:lat_1,lon_0:lon_1]
                # print(a.values)

                a = pd.DataFrame(a.values)
                a.interpolate(method="linear",inplace=True)
                a.fillna(a.mean(),inplace=True)
                a.fillna(method="bfill",inplace=True,axis=1)
                a.fillna(method="ffill",inplace=True,axis=1)

                # print(a.shape) # (25, 13)
                # print(a.values)
                a = np.array(a).flatten()  
                # print(len(a))
                # print(a)
        except:
            # print(str(date_time))
            miss_value_key = [str(date_time),np.round(lat_0.values,8),np.round(lat_1.values,8),np.round(lon_0.values,8),np.round(lon_1.values,8)]
            miss_value_key = tuple(miss_value_key)
            if miss_value_key in missing_values_u: # 键值对中有这个值
                a = missing_values_u.get(miss_value_key)
            else: # 键值对中没有这个值
                # 抽取月日
                month_day = date_time.split(" ")[0].split("-")
                month_day1 = f"{month_day[1]}{month_day[2]}" # 格式为0711这种
                month_day2 = f"{month_day[1]}-{month_day[2]}" # 格式为07-11这种
                # 抽取时间 格式为06:00:00这种
                hours = date_time.split(" ")[-1]

                # 查找对应的hycom文件
                part_filenames = parttime(filenames)
                indeices = [j for j, name in enumerate(part_filenames) if month_day1 in name] # indeices最终为一个数组表明所查找的文件在part_fimenames中的第几个位置   part_fimenames和filenames的大小是一样的

                # 存储同一月日时的文件 这里不包含年是因为要靠年去区分
                one_datas = []
                for indeic in indeices: # 只是为了和indeics作区分，并没有写成index的形式
                    name_file = filenames[indeic]
                    # print(name_file)
                    hycom_data1 = xr.open_dataset(name_file)
                    hycom_data_sst1 = hycom_data1["water_u"]
                    hycom_time1 = hycom_data1["time"].values # 标准格式
                    hycom_time2 = convert_sst(hycom_time1)

                    # 查找该文件是否存在这个时刻的数据
                    one_1 = [j for j, name in enumerate(hycom_time2) if hours in name]
                    one_2 = [j for j, name in enumerate(hycom_time2) if month_day2 in name]
                    one = list(filter(lambda x:x in one_1,one_2))

                    if one == []: # 没有这个时刻的数据，跳过
                        continue
                    else:
                        # 需要先向网格中进行插值然后再将数据append到one_datas中
                        if hycom_data_sst1["lat"].values.size == len(std_lat):
                            aa = hycom_data_sst1.loc[hycom_time1[one[0]],sel_depth[0],lat_0:lat_1,lon_0:lon_1]

                            aa = pd.DataFrame(aa.values)
                            aa.interpolate(method="linear",inplace=True)
                            aa.fillna(aa.mean(),inplace=True)
                            aa.fillna(method="bfill",inplace=True,axis=1)
                            aa.fillna(method="ffill",inplace=True,axis=1)

                            aa = np.array(aa).flatten()
                        else:
                            aa = hycom_data_sst1.loc[hycom_time1[one[0]],sel_depth[0]]
                            aa = aa.interp(lon=std_lon,lat=std_lat)
                            aa = aa.loc[lat_0:lat_1,lon_0:lon_1]

                            aa = pd.DataFrame(aa.values)
                            aa.interpolate(method="linear",inplace=True)
                            aa.fillna(aa.mean(),inplace=True)
                            aa.fillna(method="bfill",inplace=True,axis=1)
                            aa.fillna(method="ffill",inplace=True,axis=1)

                            aa = np.array(aa).flatten()
                        one_datas.append(aa)

                one_datas = np.array(one_datas)
                one_datas_mean = one_datas.mean(axis=0)

                missing_values_u[miss_value_key] = one_datas_mean
                # print(one_datas_mean)
                # a = pd.DataFrame(one_datas_mean.values)
                # a.fillna(a.mean(),inplace=True)
                # a.fillna(method="bfill",inplace=True,axis=1)

                a = np.array(one_datas_mean).flatten()
        b = np.array([str(bouydata_id),date_time])
        b = np.hstack((b,a)) # 第一个为浮标编号，第二为时间
        u_data.append(b)


        # v
        try:
            # a = sel_hycom_sst.loc[date_time,sel_depth[0],lat_0:lat_1,lon_0:lon_1] # 这一步只是为了先try出去

            # 对数据进行插值
            if sel_hycom_v["lat"].values.size == len(std_lat): # 数据的分辨率和需要插值的分辨率相同
                a = sel_hycom_v.loc[date_time,sel_depth[0],lat_0:lat_1,lon_0:lon_1]

                a = pd.DataFrame(a.values)
                a.interpolate(method="linear",inplace=True)
                a.fillna(a.mean(),inplace=True)
                a.fillna(method="bfill",inplace=True,axis=1)
                a.fillna(method="ffill",inplace=True,axis=1)

                a = np.array(a).flatten()
            else:
                # print(sel_hycom_sst["time"].values)
                a = sel_hycom_v.loc[date_time,sel_depth[0]]
                # print(a)
                a = a.interp(lon=std_lon,lat=std_lat)
                a = a.loc[lat_0:lat_1,lon_0:lon_1]
                # print(a.values)

                a = pd.DataFrame(a.values)
                a.interpolate(method="linear",inplace=True)
                a.fillna(a.mean(),inplace=True)
                a.fillna(method="bfill",inplace=True,axis=1)
                a.fillna(method="ffill",inplace=True,axis=1)

                # print(a.shape) # (25, 13)
                # print(a.values)
                a = np.array(a).flatten()  
                # print(len(a))
                # print(a)
        except:
            # print(str(date_time))
            miss_value_key = [str(date_time),np.round(lat_0.values,8),np.round(lat_1.values,8),np.round(lon_0.values,8),np.round(lon_1.values,8)]
            miss_value_key = tuple(miss_value_key)
            if miss_value_key in missing_values_v: # 键值对中有这个值
                a = missing_values_v.get(miss_value_key)
            else: # 键值对中没有这个值
                # 抽取月日
                month_day = date_time.split(" ")[0].split("-")
                month_day1 = f"{month_day[1]}{month_day[2]}" # 格式为0711这种
                month_day2 = f"{month_day[1]}-{month_day[2]}" # 格式为07-11这种
                # 抽取时间 格式为06:00:00这种
                hours = date_time.split(" ")[-1]

                # 查找对应的hycom文件
                part_filenames = parttime(filenames)
                indeices = [j for j, name in enumerate(part_filenames) if month_day1 in name] # indeices最终为一个数组表明所查找的文件在part_fimenames中的第几个位置   part_fimenames和filenames的大小是一样的

                # 存储同一月日时的文件 这里不包含年是因为要靠年去区分
                one_datas = []
                for indeic in indeices: # 只是为了和indeics作区分，并没有写成index的形式
                    name_file = filenames[indeic]
                    # print(name_file)
                    hycom_data1 = xr.open_dataset(name_file)
                    hycom_data_sst1 = hycom_data1["water_v"]
                    hycom_time1 = hycom_data1["time"].values # 标准格式
                    hycom_time2 = convert_sst(hycom_time1)

                    # 查找该文件是否存在这个时刻的数据
                    one_1 = [j for j, name in enumerate(hycom_time2) if hours in name]
                    one_2 = [j for j, name in enumerate(hycom_time2) if month_day2 in name]
                    one = list(filter(lambda x:x in one_1,one_2))

                    if one == []: # 没有这个时刻的数据，跳过
                        continue
                    else:
                        # 需要先向网格中进行插值然后再将数据append到one_datas中
                        if hycom_data_sst1["lat"].values.size == len(std_lat):
                            aa = hycom_data_sst1.loc[hycom_time1[one[0]],sel_depth[0],lat_0:lat_1,lon_0:lon_1]

                            aa = pd.DataFrame(aa.values)
                            aa.interpolate(method="linear",inplace=True)
                            aa.fillna(aa.mean(),inplace=True)
                            aa.fillna(method="bfill",inplace=True,axis=1)
                            aa.fillna(method="ffill",inplace=True,axis=1)

                            aa = np.array(aa).flatten()
                        else:
                            aa = hycom_data_sst1.loc[hycom_time1[one[0]],sel_depth[0]]
                            aa = aa.interp(lon=std_lon,lat=std_lat)
                            aa = aa.loc[lat_0:lat_1,lon_0:lon_1]

                            aa = pd.DataFrame(aa.values)
                            aa.interpolate(method="linear",inplace=True)
                            aa.fillna(aa.mean(),inplace=True)
                            aa.fillna(method="bfill",inplace=True,axis=1)
                            aa.fillna(method="ffill",inplace=True,axis=1)

                            aa = np.array(aa).flatten()
                        one_datas.append(aa)

                one_datas = np.array(one_datas)
                one_datas_mean = one_datas.mean(axis=0)

                missing_values_v[miss_value_key] = one_datas_mean
                # print(one_datas_mean)
                # a = pd.DataFrame(one_datas_mean.values)
                # a.fillna(a.mean(),inplace=True)
                # a.fillna(method="bfill",inplace=True,axis=1)

                a = np.array(one_datas_mean).flatten()
        b = np.array([str(bouydata_id),date_time])
        b = np.hstack((b,a)) # 第一个为浮标编号，第二为时间
        v_data.append(b)


------------------------1-----------------------
101
(25, 13)
102
(25, 13)
103
(25, 13)
104
(25, 13)
105
(25, 13)
106
(25, 13)
107
(25, 13)
108
(25, 13)
109
(25, 13)
110
(25, 13)
111
(25, 13)
112
(25, 13)
113
(25, 13)
114
(25, 13)
115
(25, 13)
116
(25, 13)
117
(25, 13)
118
(25, 13)
119
(25, 13)
120
(25, 13)
121
(25, 13)
122
(25, 13)
123
(25, 13)
124
(25, 13)
125
(25, 13)
126
(25, 13)
127
(25, 13)
128
(25, 13)
129
130
(25, 13)
131
(25, 13)
132
(25, 13)
133
(25, 13)
134
(25, 13)
135
(25, 13)
136
(25, 13)
137
(25, 13)
138
(25, 13)
139
(25, 13)
140
(25, 13)
141
(25, 13)
142
(25, 13)
143
(25, 13)
144
(25, 13)
145
(25, 13)
146
(25, 13)
147
(25, 13)
148
(25, 13)
149
(25, 13)
150
(25, 13)
151
(25, 13)
152
(25, 13)
153
(25, 13)
154
(25, 13)
155
(25, 13)
156
(25, 13)
157
(25, 13)
158
(25, 13)
159
(25, 13)
160
(25, 13)
161
(25, 13)
162
(25, 13)
163
(25, 13)
164
(25, 13)
165
(25, 13)
166
(25, 13)
167
(25, 13)
168
(25, 13)
169
(25, 13)
170
(25, 13)
171
(25, 13)
172
(25, 13)
173
(25, 13)
174
(25, 13

In [18]:
sst_data1 = sst_data
sal_data1 = sal_data
u_data1 = u_data
v_data1 = v_data

In [19]:
sst_data_pd = pd.DataFrame(sst_data1)
sst_data_pd

,0,1,2,3,4,5,6,7,8,9,...,317,318,319,320,321,322,323,324,325,326
0,92953,2010-12-22 12:00:00,27.30336414683949,27.325818495316938,27.36672765558416,27.405636527321555,27.44727325439453,27.4624550559304,27.47036379033869,27.47536381808194,...,27.05200056596236,26.98600058122115,26.97100032459606,26.980818661776457,26.97536381808194,26.957455028187145,26.92590938914906,26.897818652066317,26.8610914403742,26.850000381469727
1,92953,2010-12-22 18:00:00,27.71700096130371,27.84000015258789,27.8700008392334,27.83300018310547,27.796001434326172,27.773000717163086,27.759000778198242,27.735000610351562,...,27.3700008392334,27.513999938964844,27.593000411987305,27.60099983215332,27.588001251220703,27.590999603271484,27.569000244140625,27.505001068115234,27.410999298095703,27.224000930786133
2,92953,2010-12-23 00:00:00,27.72100067138672,27.829999923706055,27.849000930786133,27.80699920654297,27.76099967956543,27.729000091552734,27.711999893188477,27.690000534057617,...,27.395999908447266,27.525001525878906,27.562000274658203,27.55900001525879,27.547000885009766,27.535999298095703,27.482999801635742,27.392000198364258,27.253000259399414,27.010000228881836
3,92953,2010-12-23 06:00:00,27.658000946044922,27.80500030517578,27.87700080871582,27.874000549316406,27.822999954223633,27.770000457763672,27.73900032043457,27.722000122070312,...,27.345001220703125,27.470001220703125,27.572999954223633,27.59000015258789,27.58300018310547,27.56599998474121,27.53700065612793,27.461000442504883,27.34600067138672,27.16400146484375
4,92953,2010-12-23 12:00:00,27.402999877929688,27.555999755859375,27.697999954223633,27.77199935913086,27.759000778198242,27.71000099182129,27.672000885009766,27.655000686645508,...,27.152999877929688,27.248001098632812,27.378000259399414,27.44300079345703,27.437999725341797,27.413000106811523,27.384000778198242,27.328001022338867,27.215999603271484,27.04400062561035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107546,300534064920860,2023-10-13 12:00:00,28.665,28.735,28.696,28.511002,28.353,28.389,28.311,28.082,...,27.533,27.407001,26.912,27.318,27.739,27.637001,27.78,27.732,27.283,27.783
107547,300534064920860,2023-10-13 18:00:00,28.578,28.698,28.639,28.457,28.359001,28.327,28.282001,28.102001,...,27.372,27.23,26.717001,27.143,27.573,27.54,27.684,27.639,27.181,27.597
107548,300534064920860,2023-10-14 00:00:00,28.43,28.588001,28.637001,28.583,28.389,28.252,28.295,28.203,...,27.476,27.222,27.078001,26.562,26.978,27.426,27.437,27.666,27.57,27.096
107549,300534064920860,2023-10-14 06:00:00,28.582,28.728,28.763,28.687,28.485,28.365002,28.388,28.185001,...,27.882,27.623001,27.478,27.037,27.343,27.695,27.622,27.768002,27.713001,27.351


In [20]:
# sst_data = []
# sal_data = []
# u_data = []
# v_data = []

sst_data_pd = pd.DataFrame(sst_data1)
sst_data_pd.to_csv("/home/ly/xia_datan/wangying/trace/sst_data_surface_1.csv",index=False)
print(f"sst_data_1 successfully saved")

sal_data_pd = pd.DataFrame(sal_data1)
sal_data_pd.to_csv("/home/ly/xia_datan/wangying/trace/sal_data_surface_1.csv",index=False)
print(f"sal_data_1 successfully saved")

u_data_pd = pd.DataFrame(u_data1)
u_data_pd.to_csv("/home/ly/xia_datan/wangying/trace/u_data_surface_1.csv",index=False)
print(f"u_data_1 successfully saved")

v_data_pd = pd.DataFrame(v_data1)
v_data_pd.to_csv("/home/ly/xia_datan/wangying/trace/v_data_surface_1.csv",index=False)
print(f"v_data_1 successfully saved")

sst_data_1 successfully saved
sal_data_1 successfully saved
u_data_1 successfully saved
v_data_1 successfully saved


In [21]:
len(sst_data_pd)

107551

In [ ]:
# from scipy import interpolate
# for i in range(322,323):
#     a = sst_data_pd.loc[i]
#     
#     a1 = a[:2]
#     a2 = a[2:].values
#     
#     a2 = a2.reshape(25,13)
#     # print(a2)
#     for j in range(len(a2)):
#         print(a2[j])
    
    # a2_0 = a2[0]
    # 
    # # a3 = np.array(a2_0,dtype='float')
    # # print(a3)
    # 
    # # 创建插值函数
    # interpolator = interpolate.interp2d(np.arange(a2.shape[1]), np.arange(a2.shape[0]), a2, kind='linear', fill_value='extrapolate')
    # 
    # # 生成插值后的数组
    # interpolated_data = interpolator(np.arange(a2.shape[1]), np.arange(a2.shape[0]))
    # 
    # print(interpolated_data)


# for i in range(55,56):
#     a = sst_data_pd.loc[i].values
#     a = np.array(a)
#     if np.any(a == "nan"):


    